In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix, roc_curve, auc, precision_recall_curve, roc_auc_score
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pickle
import seaborn as sns

print('Libraries imported successfully!')
print(f'PyTorch version: {torch.__version__}')
print(f'Pandas version: {pd.__version__}')
print(f'Numpy version: {np.__version__}')
print(f'Sklearn version: {np.__version__}')
print(f'Matplotlib version: {np.__version__}')
print(f'scipy version: {np.__version__}')
print(f'tqdm version: {np.__version__}')
print(f'pickle version: {np.__version__}')
print(f'pathlib version: {np.__version__}')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {DEVICE}')

############################
# Other Parameters
############################
MODEL_DIR = Path('model_weights_4')
info_path = MODEL_DIR / 'training_info.pkl'
csv_path = MODEL_DIR / 'test_results.csv'
txt_path = MODEL_DIR /'test_results.txt'
png_path = MODEL_DIR / 'result.png'

distribution_png_path = MODEL_DIR / 'anomaly_score_distribution.png'
boxplot_png_path = MODEL_DIR /  'anomaly_score_boxplot.png'
precision_recall_png_path = MODEL_DIR / 'precision_recall_curve.png'
roc_curve_png_path = MODEL_DIR / 'roc_curve.png'
latent_pca_png_path = MODEL_DIR / 'latent_pca_plot.png'

WORKER_NODES = 20 # Should be less than the number of threads in your CPU
ANOMALY_THRESHOLD = 0.8

############################
# Hyperparameters
############################
# Number of neurons in the hidden layer of the first-layer VAE encoder.
# Affects the model's capacity to learn representations from the input data.
FIRST_LAYER_HIDDEN_SIZE = 8

# Number of neurons in the hidden layer of the second-layer VAE encoder.
# Influences how well the model can capture patterns from the subspace errors produced by the first layer.
SECOND_LAYER_HIDDEN_SIZE = 32

# Dimensionality of the latent space in the first-layer VAE.
# Lower values force the model to learn compressed representations, which can enhance anomaly detection by focusing on essential features.
LATENT_DIM_FIRST_LAYER = 4

# Dimensionality of the latent space in the second-layer VAE.
# Determines the level of abstraction in encoding the errors from the first layer; impacts the model's ability to detect anomalies.
LATENT_DIM_SECOND_LAYER = 64

# Threshold used for clustering latent representations.
# Controls the sensitivity of the model in distinguishing between normal and anomalous data points.
CLUSTER_THRESHOLD = 0.6

# Number of samples processed before the model parameters are updated.
# Larger batch sizes can lead to more stable gradient estimates but require more memory.
BATCH_SIZE = 512

# Number of training iterations over the entire dataset for the first-layer VAE.
# More epochs can improve learning but may increase the risk of overfitting.
EPOCHS_FIRST_LAYER = 5

# Number of training iterations over the entire dataset for the second-layer VAE.
# Affects the model's ability to learn from the aggregated subspace errors.
EPOCHS_SECOND_LAYER = 6

# Step size for the optimizer during training.
# A smaller learning rate can lead to more precise convergence but may slow down training.
LEARNING_RATE = 1e-5

# Maximum allowed norm of gradients during training.
# Used for gradient clipping to prevent exploding gradients and stabilize training.
MAX_GRAD_NORM = 5.0

# Lower bound for clamping the logarithm of the variance in the VAE.
# Prevents the variance from becoming too small, which can cause numerical instability.
CLAMP_LOGVAR_LOW = -10

# Upper bound for clamping the logarithm of the variance in the VAE.
# Prevents the variance from becoming too large, which can also cause instability.
CLAMP_LOGVAR_HIGH = 10

Libraries imported successfully!
PyTorch version: 2.5.1
Pandas version: 2.2.3
Numpy version: 1.26.4
Sklearn version: 1.26.4
Matplotlib version: 1.26.4
scipy version: 1.26.4
tqdm version: 1.26.4
pickle version: 1.26.4
pathlib version: 1.26.4
Using device: cuda


In [2]:
############################
# Load and Preprocess Data
############################
data = pd.read_csv('ARP_MitM_dataset.csv', header=None).values
labels = pd.read_csv('ARP_MitM_labels_Y.csv', header=None).values.flatten()

# The first million are guaranteed normal (label=0)
TRAIN_SIZE = 1_000_000

print("Checking dataset for NaNs and Infs...")
if np.isnan(data).any():
    print("WARNING: NaNs found in the dataset!")
else:
    print("No NaNs in dataset.")
if np.isinf(data).any():
    print("WARNING: Infs found in the dataset!")
else:
    print("No Infs in dataset.")

# Fit scaler on the training portion
train_data = data[:TRAIN_SIZE]
train_labels = labels[:TRAIN_SIZE]

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)

# The test portion
test_data = data[TRAIN_SIZE:]
test_data = scaler.transform(test_data)
test_labels = labels[TRAIN_SIZE:]

print("Data stats after scaling (train data):")
print("max:", train_data.max(), "min:", train_data.min(), "mean:", train_data.mean(), "std:", train_data.std())



Checking dataset for NaNs and Infs...
No NaNs in dataset.
No Infs in dataset.
Data stats after scaling (train data):
max: 271.19894087275117 min: -36.578873736231806 mean: -2.1953323796523087e-16 std: 0.9999999999999949


In [ ]:
############################
# Feature Mapping (Clustering)
############################
# corr_matrix = np.corrcoef(train_data, rowvar=False)
# distance_matrix = 1 - np.abs(corr_matrix)
# condensed_distance = squareform(distance_matrix, checks=False)
# Z = linkage(condensed_distance, method='complete')
# cluster_assignments = fcluster(Z, t=CLUSTER_THRESHOLD, criterion='distance')

# feature_groups_dict = {}
# for idx, cluster_id in enumerate(cluster_assignments):
#     if cluster_id not in feature_groups_dict:
#         feature_groups_dict[cluster_id] = []
#     feature_groups_dict[cluster_id].append(idx)

# feature_groups = [feature_groups_dict[key] for key in sorted(feature_groups_dict.keys())]
# print(f'Number of feature groups: {len(feature_groups)}')
# # pretty print the feature groups
# for idx, group in enumerate(feature_groups):
#     print(f'Group {idx}: {group}')
feature_groups = [
    [85, 86],  # Group 0
    [92, 93],  # Group 1
    [56, 63],  # Group 2
    [36, 43, 50],  # Group 3
    [35, 42, 49],  # Group 4
    [66, 69, 72, 75, 78],  # Group 5
    [67, 70, 73, 76, 79],  # Group 6
    [106, 107],  # Group 7
    [113, 114],  # Group 8
    [33, 40, 47, 54, 61, 83, 90, 97, 104, 111],  # Group 9
    [57, 64],  # Group 10
    [0, 2, 3, 5, 6, 8, 9, 11, 14, 15, 17, 18, 20, 21, 23, 24, 26, 29, 30, 32, 34, 37, 39, 41, 44, 46, 48, 51, 53, 55, 60, 62, 65, 68, 71, 74, 82, 84, 89, 91, 96, 98, 103, 105, 110, 112],  # Group 11
    [99, 100],  # Group 12
    [12, 27, 58, 77, 108],  # Group 13
    [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 38, 45, 52, 59, 80, 81, 87, 88, 94, 95, 101, 102, 109]  # Group 14
]
print(f'Number of feature groups: {len(feature_groups)}')
for idx, group in enumerate(feature_groups):
    print(f'Group {idx}: {group}')

Number of feature groups: 15
Group 0: [85, 86]
Group 1: [92, 93]
Group 2: [56, 63]
Group 3: [36, 43, 50]
Group 4: [35, 42, 49]
Group 5: [66, 69, 72, 75, 78]
Group 6: [67, 70, 73, 76, 79]
Group 7: [106, 107]
Group 8: [113, 114]
Group 9: [33, 40, 47, 54, 61, 83, 90, 97, 104, 111]
Group 10: [57, 64]
Group 11: [0, 2, 3, 5, 6, 8, 9, 11, 14, 15, 17, 18, 20, 21, 23, 24, 26, 29, 30, 32, 34, 37, 39, 41, 44, 46, 48, 51, 53, 55, 60, 62, 65, 68, 71, 74, 82, 84, 89, 91, 96, 98, 103, 105, 110, 112]
Group 12: [99, 100]
Group 13: [12, 27, 58, 77, 108]
Group 14: [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 38, 45, 52, 59, 80, 81, 87, 88, 94, 95, 101, 102, 109]


In [4]:
############################
# Dataset Definitions
############################
class FirstLayerDataset(Dataset):
    def __init__(self, data, features):
        self.X = data[:, features]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32)


class SecondLayerDataset(Dataset):
    def __init__(self, latent_data):
        self.X = latent_data

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32)

############################
# VAE Model Definition
############################
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        self.fc2 = nn.Linear(latent_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, input_dim)
        self.relu = nn.ReLU()

        # Weight initialization
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0.0)

    def encode(self, x):
        h = self.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        logvar = torch.clamp(logvar, CLAMP_LOGVAR_LOW, CLAMP_LOGVAR_HIGH)
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std, logvar

    def decode(self, z):
        h = self.relu(self.fc2(z))
        return self.fc3(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z, logvar = self.reparameterize(mu, logvar)
        recon_x = self.decode(z)
        return recon_x, mu, logvar

def vae_loss(recon_x, x, mu, logvar):
    recon_loss = nn.functional.mse_loss(recon_x, x, reduction='mean')
    logvar = torch.clamp(logvar, CLAMP_LOGVAR_LOW, CLAMP_LOGVAR_HIGH)
    kld_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - torch.exp(logvar))
    return recon_loss + kld_loss


In [5]:
############################
# Train First-Layer VAEs
############################
# Check if models already exist
def all_vae_weights_exist(num_vaes):
    # Check all first layer VAE files
    for idx in range(num_vaes):
        vae_path = MODEL_DIR / f'first_layer_vae_{idx}.pth'
        if not vae_path.is_file():
            return False
    return True


first_layer_vaes = []
if MODEL_DIR.is_dir() and info_path.is_file():
    # Attempt to load existing weights and info
    with open(info_path, 'rb') as f:
        saved_info = pickle.load(f)
    saved_feature_groups = saved_info['feature_groups']
    saved_scaler = saved_info['scaler']
    saved_first_layer_hidden_size = saved_info['first_layer_hidden_size']
    saved_latent_dim_first_layer = saved_info['latent_dim_first_layer']

    # Check if all weights exist
    if all_vae_weights_exist(len(saved_feature_groups)):
        print("Found existing first-layer VAEs, loading them...")
        for idx, features in enumerate(saved_feature_groups):
            input_dim = len(features)
            vae = VAE(input_dim, saved_first_layer_hidden_size, saved_latent_dim_first_layer).to(DEVICE)
            vae.load_state_dict(torch.load(MODEL_DIR / f'first_layer_vae_{idx}.pth', map_location=DEVICE))
            vae.eval()
            first_layer_vaes.append(vae)
        # Use saved_feature_groups and saved_scaler as they are loaded from info
        feature_groups = saved_feature_groups
        scaler = saved_scaler
    else:
        # Some weights missing, retrain
        print("Weights missing, retraining first-layer VAEs...")
        os.makedirs(MODEL_DIR, exist_ok=True)

        # Train as usual
        feature_groups = saved_feature_groups  # we can still use these if needed
        # If feature_groups aren't set anywhere else (like from a previous run), 
        # you'd need to run the clustering process again before training.

        first_layer_vaes = []
        for idx, features in enumerate(feature_groups):
            input_dim = len(features)
            vae = VAE(input_dim, FIRST_LAYER_HIDDEN_SIZE, LATENT_DIM_FIRST_LAYER).to(DEVICE)
            optimizer = torch.optim.Adam(vae.parameters(), lr=LEARNING_RATE)

            train_dataset = FirstLayerDataset(train_data, features)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

            vae.train()
            for epoch in range(EPOCHS_FIRST_LAYER):
                total_loss = 0
                for i, batch in enumerate(train_loader):
                    batch = batch.to(DEVICE)
                    optimizer.zero_grad()
                    recon_x, mu, logvar = vae(batch)
                    loss = vae_loss(recon_x, batch, mu, logvar)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(vae.parameters(), MAX_GRAD_NORM)
                    optimizer.step()
                    total_loss += loss.item()
                avg_loss = total_loss / len(train_loader.dataset)
                print(f'First Layer VAE {idx+1}/{len(feature_groups)} Epoch [{epoch+1}/{EPOCHS_FIRST_LAYER}] Loss: {avg_loss:.4f}')

            vae.eval()
            first_layer_vaes.append(vae)

        # Save after training
        for idx, vae in enumerate(first_layer_vaes):
            vae_path = MODEL_DIR / f'first_layer_vae_{idx}.pth'
            torch.save(vae.state_dict(), vae_path)
            print(f"Saved {vae_path}")

        info_dict = {
            'feature_groups': feature_groups,
            'scaler': scaler,
            'train_size': TRAIN_SIZE,
            'input_dim': data.shape[1],
            'first_layer_hidden_size': FIRST_LAYER_HIDDEN_SIZE,
            'latent_dim_first_layer': LATENT_DIM_FIRST_LAYER
        }

        with open(info_path, 'wb') as f:
            pickle.dump(info_dict, f)
        print("Saved feature groups, scaler, and configuration info.")

else:
    # No model directory or info found, we must train from scratch
    print("No existing models found, training first-layer VAEs...")
    os.makedirs(MODEL_DIR, exist_ok=True)

    # Run your feature mapping and set up feature_groups and scaler as before
    # (Assuming feature_groups and scaler are already determined by now)
    # feature_groups = ...
    # scaler = ...

    first_layer_vaes = []
    for idx, features in enumerate(feature_groups):
        input_dim = len(features)
        vae = VAE(input_dim, FIRST_LAYER_HIDDEN_SIZE, LATENT_DIM_FIRST_LAYER).to(DEVICE)
        optimizer = torch.optim.Adam(vae.parameters(), lr=LEARNING_RATE)

        train_dataset = FirstLayerDataset(train_data, features)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

        vae.train()
        for epoch in range(EPOCHS_FIRST_LAYER):
            total_loss = 0
            for i, batch in enumerate(train_loader):
                batch = batch.to(DEVICE)
                optimizer.zero_grad()
                recon_x, mu, logvar = vae(batch)
                loss = vae_loss(recon_x, batch, mu, logvar)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(vae.parameters(), MAX_GRAD_NORM)
                optimizer.step()
                total_loss += loss.item()
            avg_loss = total_loss / len(train_loader.dataset)
            print(f'First Layer VAE {idx+1}/{len(feature_groups)} Epoch [{epoch+1}/{EPOCHS_FIRST_LAYER}] Loss: {avg_loss:.4f}')

        vae.eval()
        first_layer_vaes.append(vae)

    # Save after training
    for idx, vae in enumerate(first_layer_vaes):
        vae_path = MODEL_DIR / f'first_layer_vae_{idx}.pth'
        torch.save(vae.state_dict(), vae_path)
        print(f"Saved {vae_path}")

    info_dict = {
        'feature_groups': feature_groups,
        'scaler': scaler,
        'train_size': TRAIN_SIZE,
        'input_dim': data.shape[1],
        'first_layer_hidden_size': FIRST_LAYER_HIDDEN_SIZE,
        'latent_dim_first_layer': LATENT_DIM_FIRST_LAYER
    }

    with open(info_path, 'wb') as f:
        pickle.dump(info_dict, f)
    print("Saved feature groups, scaler, and configuration info.")

# first_layer_vaes now loaded or trained and ready for next steps

No existing models found, training first-layer VAEs...
First Layer VAE 1/15 Epoch [1/5] Loss: 0.0162
First Layer VAE 1/15 Epoch [2/5] Loss: 0.0107
First Layer VAE 1/15 Epoch [3/5] Loss: 0.0073
First Layer VAE 1/15 Epoch [4/5] Loss: 0.0051
First Layer VAE 1/15 Epoch [5/5] Loss: 0.0039
First Layer VAE 2/15 Epoch [1/5] Loss: 0.0046
First Layer VAE 2/15 Epoch [2/5] Loss: 0.0036
First Layer VAE 2/15 Epoch [3/5] Loss: 0.0031
First Layer VAE 2/15 Epoch [4/5] Loss: 0.0026
First Layer VAE 2/15 Epoch [5/5] Loss: 0.0023
First Layer VAE 3/15 Epoch [1/5] Loss: 0.0023
First Layer VAE 3/15 Epoch [2/5] Loss: 0.0021
First Layer VAE 3/15 Epoch [3/5] Loss: 0.0019
First Layer VAE 3/15 Epoch [4/5] Loss: 0.0018
First Layer VAE 3/15 Epoch [5/5] Loss: 0.0017
First Layer VAE 4/15 Epoch [1/5] Loss: 0.0447
First Layer VAE 4/15 Epoch [2/5] Loss: 0.0341
First Layer VAE 4/15 Epoch [3/5] Loss: 0.0256
First Layer VAE 4/15 Epoch [4/5] Loss: 0.0191
First Layer VAE 4/15 Epoch [5/5] Loss: 0.0145
First Layer VAE 5/15 Epoc

In [ ]:
latent_vectors_path =  MODEL_DIR + 'all_latent_vectors_train.npy'

if os.path.isfile(latent_vectors_path):
    print(f"Found {latent_vectors_path}, loading latent vectors...")
    all_latent_vectors_train = np.load(latent_vectors_path)
    print("Loaded all_latent_vectors_train from disk.")
else:
    print(f"{latent_vectors_path} not found, extracting latent vectors from first-layer VAEs...")

    def process_sample(sample, first_layer_vaes, feature_groups, device):
        """
        Process a single training sample to extract its latent representation
        from the first-layer VAEs.
        """
        sample_latents = []
        with torch.no_grad():
            for vae, features in zip(first_layer_vaes, feature_groups):
                x_sub = torch.tensor(sample[features], dtype=torch.float32, device=device).unsqueeze(0)
                mu, logvar = vae.encode(x_sub)
                z, _ = vae.reparameterize(mu, logvar)
                sample_latents.append(z.squeeze(0).cpu().numpy())
        return np.concatenate(sample_latents, axis=0)

    results = [None] * len(train_data)  # Pre-allocate a list for results

    # Adjust max_workers as needed based on your system's capabilities
    max_workers = 40

    print("Extracting latent vectors from first-layer VAEs in parallel...")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_sample, train_data[i], first_layer_vaes, feature_groups, DEVICE): i 
                   for i in range(len(train_data))}

        # Use tqdm to show progress
        for f in tqdm(as_completed(futures), total=len(futures), desc="Extracting Latents"):
            i = futures[f]  # Retrieve the index associated with this future
            results[i] = f.result()

    all_latent_vectors_train = np.array(results)
    print("Latent vector extraction completed.")

    # Save the latent vectors for future runs
    os.makedirs('model_weights', exist_ok=True)
    np.save(latent_vectors_path, all_latent_vectors_train)
    print(f"all_latent_vectors_train saved to '{latent_vectors_path}'")


Found model_weights/all_latent_vectors_train.npy, loading latent vectors...
Loaded all_latent_vectors_train from disk.


In [7]:
############################
# Train Second-Layer VAE on Latent Representations
############################
second_layer_input_dim = all_latent_vectors_train.shape[1]
second_layer_vae = VAE(second_layer_input_dim, SECOND_LAYER_HIDDEN_SIZE, LATENT_DIM_SECOND_LAYER).to(DEVICE)
second_optimizer = torch.optim.Adam(second_layer_vae.parameters(), lr=LEARNING_RATE)

second_dataset = SecondLayerDataset(all_latent_vectors_train)
second_loader = DataLoader(second_dataset, batch_size=BATCH_SIZE, shuffle=True)

second_layer_vae.train()
for epoch in range(EPOCHS_SECOND_LAYER):
    total_loss = 0
    for i, batch in enumerate(second_loader):
        batch = batch.to(DEVICE)
        second_optimizer.zero_grad()
        recon_x, mu, logvar = second_layer_vae(batch)
        loss = vae_loss(recon_x, batch, mu, logvar)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(second_layer_vae.parameters(), MAX_GRAD_NORM)
        second_optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(second_loader.dataset)
    print(f'Second Layer Epoch [{epoch+1}/{EPOCHS_SECOND_LAYER}] Loss: {avg_loss:.4f}')

second_layer_vae.eval()

Second Layer Epoch [1/6] Loss: 0.0083
Second Layer Epoch [2/6] Loss: 0.0068
Second Layer Epoch [3/6] Loss: 0.0060
Second Layer Epoch [4/6] Loss: 0.0055
Second Layer Epoch [5/6] Loss: 0.0051
Second Layer Epoch [6/6] Loss: 0.0048


VAE(
  (fc1): Linear(in_features=60, out_features=32, bias=True)
  (fc_mu): Linear(in_features=32, out_features=64, bias=True)
  (fc_logvar): Linear(in_features=32, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=60, bias=True)
  (relu): ReLU()
)

In [8]:


def get_anomaly_score(packet_scaled):
    # Extract latents from first-layer VAEs
    latents = []
    with torch.no_grad():
        for vae, features in zip(first_layer_vaes, feature_groups):
            x_sub = torch.tensor(packet_scaled[features], dtype=torch.float32, device=DEVICE).unsqueeze(0)
            mu, logvar = vae.encode(x_sub)
            z, _ = vae.reparameterize(mu, logvar)
            latents.append(z.squeeze(0))
    latents = torch.cat(latents).unsqueeze(0).to(DEVICE)

    # Pass through second-layer VAE
    with torch.no_grad():
        recon_x, mu, logvar = second_layer_vae(latents)
        loss = vae_loss(recon_x, latents, mu, logvar)
    return loss.item()

def process_test_packet(i):
    packet = test_data[i]
    score = get_anomaly_score(packet)
    return i, score

############################
# Check if test_results.csv already exists
############################
if os.path.exists(csv_path):
    # If test_results.csv exists, load test_scores from it and do not recompute
    print(f"Found {csv_path}, loading test_scores from disk...")
    test_scores = np.loadtxt(csv_path, delimiter=',')
    print("test_scores loaded from test_results.csv")
else:
    # If test_results.csv does not exist, compute test_scores and save them
    test_scores = [None] * len(test_data)
    print("Processing test data in parallel to compute test_scores...")
    with ThreadPoolExecutor(max_workers=WORKER_NODES) as executor:
        futures = {executor.submit(process_test_packet, i): i for i in range(len(test_data))}
        for f in tqdm(as_completed(futures), total=len(futures), desc="Scoring Test Packets"):
            i, score = f.result()
            test_scores[i] = score

    test_scores = np.array(test_scores)
    # Save test_scores to CSV
    np.savetxt(csv_path, test_scores, delimiter=',')
    print(f"test_scores saved to {csv_path}")

# Ensure test_labels are int
test_labels = test_labels.astype(int)

############################
# Visualize Anomaly Score Distributions
############################
# Separate scores by label
normal_scores = test_scores[test_labels == 0]
malicious_scores = test_scores[test_labels == 1]

# Plot Histograms
plt.figure(figsize=(10,6))
sns.histplot(normal_scores, bins=100, color='blue', alpha=0.5, label='Normal', kde=True)
sns.histplot(malicious_scores, bins=100, color='red', alpha=0.5, label='Malicious', kde=True)
plt.axvline(x=ANOMALY_THRESHOLD, color='green', linestyle='--', label=f'Threshold: {ANOMALY_THRESHOLD:.4f}')
plt.xlabel('Anomaly Score')
plt.ylabel('Number of Packets')
plt.title('Distribution of Anomaly Scores for Test Packets')
plt.legend()
plt.tight_layout()
plt.savefig(distribution_png_path)
plt.close()
print(f"Anomaly score distribution plot saved to '{distribution_png_path}'")

# Plot Boxplot
df_scores = pd.DataFrame({
    'Anomaly Score': test_scores,
    'Label': ['Normal' if label == 0 else 'Malicious' for label in test_labels]
})

plt.figure(figsize=(8,6))
sns.boxplot(x='Label', y='Anomaly Score', data=df_scores, palette={'Normal': 'blue', 'Malicious': 'red'})
plt.title('Boxplot of Anomaly Scores by Label')
plt.tight_layout()
plt.savefig(boxplot_png_path)
plt.close()
print(f"Anomaly score boxplot saved to '{boxplot_png_path}'")

############################
# Compute and Save Metrics and Plot if Necessary
############################
# Only compute metrics and generate plot if test_results.txt and result.png do not exist
if not os.path.exists(txt_path) and not os.path.exists(png_path):
    print("Computing metrics and generating plot since neither test_results.txt nor result.png exist...")
    
    # Compute FPR, TPR, and thresholds for ROC curve
    fpr_vals, tpr_vals, roc_thresholds = roc_curve(test_labels, test_scores)

    # Define target FPRs
    target_fprs = [0.001, 0.01]
    thresholds_at_target_fprs = {}

    for target_fpr in target_fprs:
        # Find the index where FPR is closest to the target FPR
        idx = np.argmin(np.abs(fpr_vals - target_fpr))
        ANOMALY_THRESHOLD = roc_thresholds[idx]
        thresholds_at_target_fprs[target_fpr] = ANOMALY_THRESHOLD
        print(f"Anomaly Threshold for FPR={target_fpr}: {ANOMALY_THRESHOLD:.4f}")

        # Compute predictions using the threshold
        y_pred = (test_scores >= ANOMALY_THRESHOLD).astype(int)

        # Compute Metrics
        acc = accuracy_score(test_labels, y_pred)
        prec = precision_score(test_labels, y_pred, zero_division=0)
        rec = recall_score(test_labels, y_pred, zero_division=0)
        f1 = f1_score(test_labels, y_pred, zero_division=0)

        # Compute confusion matrix
        tn, fp, fn, tp = confusion_matrix(test_labels, y_pred).ravel()

        # Calculate TPR and FNR
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        fnr = fn / (tp + fn) if (tp + fn) > 0 else 0

        print(f"Metrics for FPR={target_fpr}:")
        print(f"Accuracy: {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall: {rec:.4f}")
        print(f"F1-score: {f1:.4f}")
        print(f"True Positive Rate (TPR): {tpr:.4f}")
        print(f"False Negative Rate (FNR): {fnr:.4f}")


    # Compute final predictions with the best threshold
    y_pred = (test_scores >= ANOMALY_THRESHOLD).astype(int)

    # Compute Metrics
    acc = accuracy_score(test_labels, y_pred)
    prec = precision_score(test_labels, y_pred, zero_division=0)
    rec = recall_score(test_labels, y_pred, zero_division=0)
    f1 = f1_score(test_labels, y_pred, zero_division=0)

    # Compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(test_labels, y_pred).ravel()

    # Calculate TPR and FNR
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fnr = fn / (tp + fn) if (tp + fn) > 0 else 0

    print(f"True Positive Rate (TPR): {tpr:.4f}")
    print(f"False Negative Rate (FNR): {fnr:.4f}")

    # Save metrics and model parameters to test_results.txt
    with open(txt_path, 'w') as f:
        f.write("Model Parameters:\n")
        f.write(f"FIRST_LAYER_HIDDEN_SIZE: {FIRST_LAYER_HIDDEN_SIZE}\n")
        f.write(f"SECOND_LAYER_HIDDEN_SIZE: {SECOND_LAYER_HIDDEN_SIZE}\n")
        f.write(f"LATENT_DIM_FIRST_LAYER: {LATENT_DIM_FIRST_LAYER}\n")
        f.write(f"LATENT_DIM_SECOND_LAYER: {LATENT_DIM_SECOND_LAYER}\n")
        f.write(f"CLUSTER_THRESHOLD: {CLUSTER_THRESHOLD}\n")
        f.write(f"BATCH_SIZE: {BATCH_SIZE}\n")
        f.write(f"EPOCHS_FIRST_LAYER: {EPOCHS_FIRST_LAYER}\n")
        f.write(f"EPOCHS_SECOND_LAYER: {EPOCHS_SECOND_LAYER}\n")
        f.write(f"LEARNING_RATE: {LEARNING_RATE}\n")
        f.write(f"MAX_GRAD_NORM: {MAX_GRAD_NORM}\n")
        f.write(f"CLAMP_LOGVAR_LOW: {CLAMP_LOGVAR_LOW}\n")
        f.write(f"CLAMP_LOGVAR_HIGH: {CLAMP_LOGVAR_HIGH}\n")
        f.write("\n")
        f.write("Metrics:\n")
        f.write(f"Anomaly Threshold: {ANOMALY_THRESHOLD}\n")
        f.write(f"Number of Test Packets: {len(test_labels)}\n")
        f.write("\n")
        f.write(f"Accuracy: {acc:.4f}\n")
        f.write(f"Precision: {prec:.4f}\n")
        f.write(f"Recall: {rec:.4f}\n")
        f.write(f"F1-score: {f1:.4f}\n")
        f.write(f"True Positive Rate (TPR): {tpr:.4f}\n")
        f.write(f"False Negative Rate (FNR): {fnr:.4f}\n")
    print(f"Metrics saved to '{txt_path}'")

    # Generate and save the plot to result.png
    plt.figure(figsize=(10,6))
    plt.scatter(np.arange(len(test_scores))[test_labels==0], test_scores[test_labels==0], s=1, c='blue', label='Normal')
    plt.scatter(np.arange(len(test_scores))[test_labels==1], test_scores[test_labels==1], s=1, c='red', label='Malicious')
    plt.axhline(y=ANOMALY_THRESHOLD, color='green', linestyle='--', label=f'Threshold: {ANOMALY_THRESHOLD:.4f}')
    plt.xlabel('Packet Index (Test Set)')
    plt.ylabel('Anomaly Score')
    plt.title('Anomaly Scores on Test Packets')
    plt.legend()
    plt.tight_layout()
    plt.savefig(png_path)
    plt.close()
    print(f"Plot saved to '{png_path}'")

    # Plot Precision-Recall Curve
    precision_vals, recall_vals, pr_thresholds = precision_recall_curve(test_labels, test_scores)
    f1_vals = 2 * (precision_vals * recall_vals) / (precision_vals + recall_vals + 1e-8)
    best_pr_idx = np.argmax(f1_vals)
    best_pr_threshold = pr_thresholds[best_pr_idx]
    best_pr_f1 = f1_vals[best_pr_idx]

    plt.figure(figsize=(10,6))
    plt.plot(recall_vals, precision_vals, marker='.', label='Precision-Recall Curve')
    plt.scatter(recall_vals[best_pr_idx], precision_vals[best_pr_idx], color='red', label=f'Best Threshold: {best_pr_threshold:.4f}\nF1-score: {best_pr_f1:.4f}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()
    plt.tight_layout()
    plt.savefig(precision_recall_png_path)
    plt.close()
    print(f"Precision-Recall curve saved to '{precision_recall_png_path}'")

    # Plot ROC Curve
    fpr_vals, tpr_vals, roc_thresholds = roc_curve(test_labels, test_scores)
    roc_auc = roc_auc_score(test_labels, test_scores)

    # Find the best threshold (closest to top-left)
    distances = np.sqrt((1 - tpr_vals)**2 + fpr_vals**2)
    best_roc_idx = np.argmin(distances)
    best_roc_threshold = roc_thresholds[best_roc_idx]

    plt.figure(figsize=(10,6))
    plt.plot(fpr_vals, tpr_vals, label=f'ROC Curve (AUC = {roc_auc:.4f})')
    plt.scatter(fpr_vals[best_roc_idx], tpr_vals[best_roc_idx], color='red', label=f'Best Threshold: {best_roc_threshold:.4f}')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.tight_layout()
    plt.savefig(roc_curve_png_path)
    plt.close()
    print(f"ROC curve saved to '{roc_curve_png_path}'")

    # Optionally, save a PCA plot of latent representations
    from sklearn.decomposition import PCA

    # Reduce dimensionality for visualization (only if computationally feasible)
    print("Generating PCA plot for latent representations...")
    pca = PCA(n_components=2)
    latent_reduced = pca.fit_transform(all_latent_vectors_train)

    # For test set
    test_latent_vectors = []
    with torch.no_grad():
        for i in tqdm(range(len(test_data)), desc="Extracting Test Latents"):
            packet = test_data[i]
            latents = []
            for vae, features in zip(first_layer_vaes, feature_groups):
                x_sub = torch.tensor(packet[features], dtype=torch.float32, device=DEVICE).unsqueeze(0)
                mu, logvar = vae.encode(x_sub)
                z, _ = vae.reparameterize(mu, logvar)
                latents.append(z.squeeze(0).cpu().numpy())
            latents = np.concatenate(latents, axis=0)
            test_latent_vectors.append(latents)

    test_latent_vectors = np.array(test_latent_vectors)
    test_latent_reduced = pca.transform(test_latent_vectors)

    # Create a DataFrame for plotting
    df_pca = pd.DataFrame({
        'PC1': test_latent_reduced[:,0],
        'PC2': test_latent_reduced[:,1],
        'Label': ['Normal' if label == 0 else 'Malicious' for label in test_labels]
    })

    # Plot PCA
    plt.figure(figsize=(10,6))
    sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='Label', palette={'Normal': 'blue', 'Malicious': 'red'}, s=10, alpha=0.5)
    plt.title('PCA of Test Latent Representations')
    plt.legend(title='Label', labels=['Normal', 'Malicious'])
    plt.tight_layout()
    plt.savefig(latent_pca_png_path)
    plt.close()
    print(f"PCA plot of latent representations saved to '{latent_pca_png_path}'")

else:
    print("Not updating metrics or plot because test_results.txt or result.png (or both) already exist.")

Processing test data in parallel to compute test_scores...


: 